In [1]:
suppressMessages(library(ggplot2))
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(purrr))
suppressMessages(library(readr))
suppressMessages(library(patchwork))
suppressMessages(library(BSgenome.Hsapiens.UCSC.hg38))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(ggrepel))
suppressMessages(library(rhdf5))
suppressMessages(library(chromVARmotifs))
suppressMessages(library(tidyr))
suppressMessages(library(Ipaper))
suppressMessages(library(ComplexHeatmap))
library(colorRamp2)

In [2]:
set.seed(42)
addArchRThreads(threads = 64)

Setting default number of Parallel threads to 64.



#### 上皮

In [3]:
proj <- loadArchRProject("./ArchRProject_epithelial", showLogo = FALSE)

Successfully loaded ArchRProject!



In [4]:
proj$neoadjuvant <- ifelse(
  proj$neoadjuvant == "after_neoadjuvant", 
  "Post-NAT", 
  "Non-NAT"
)

In [5]:
neo_after <- getCellNames(ArchRProj = proj)[proj$Sample %ni% c("12N", "10N")]
proj <- proj[neo_after,]

In [ ]:
VarMotifs <- getVarDeviations(proj, name = "homerMatrix", plot = F)
MotifScoreClusters <- getMarkerFeatures(proj, 
                                        useMatrix = "homerMatrix", 
                                        groupBy = "neoadjuvant",
                                        bias = c("TSSEnrichment", "log10(nFrags)"),
                                        testMethod = "wilcoxon", 
                                        useSeqnames = "z")

fh <- function(x) hclust(dist(x), method="ward.D2")
idy <- which(mcols(MotifScoreClusters)$name %in% VarMotifs$name[c(1:50)])
row_means <- rowMeans(mtx2)   # 每个motif的平均score
mtx2 <- mtx2[order(row_means, decreasing = TRUE), ]   # 从大到小排序
mtx2 <- assays(MotifScoreClusters[idy,])$Mean %>% `rownames<-`(., mcols(MotifScoreClusters)$name[idy])

col_fun2 <- colorRamp2(c(-5,-2.5,0,2,5), paletteContinuous(set = "solarExtra", n = 5))
ht2 <- Heatmap(mtx2, name = "Mean motif score", cluster_rows = fh, cluster_columns = F, show_row_dend = F, col = col_fun2)
options(repr.plot.width=10, repr.plot.height=10)
p6 <- draw(ht2)
plotPDF(p6, name = "epi-Motif-Heatmap", width = 6, height = 8, ArchRProj = proj, addDOC = FALSE)

In [ ]:
# ===== Packages =====
# 按motif score 得分进行过滤的代码
suppressPackageStartupMessages({
  library(ArchR)
  library(ComplexHeatmap)
  library(circlize)
  library(magrittr)
})

# ===== 用户可调参数 =====
top_n_var_motifs <- 50     # 从 VarMotifs 里取前多少个变异度最高的 motif
threshold <- -2             # 筛选阈值（对行均值）
use_abs <- FALSE           # 是否用绝对值筛选：TRUE 表示 |mean| > threshold
decreasing_order <- TRUE   # 排序方向：TRUE 从高到低；FALSE 从低到高

# ===== 载入项目并计算所需对象 =====
# 假设 proj 已经存在。如需加载，请取消下一行注释并设置路径
# proj <- loadArchRProject("./ArchRProject", showLogo = FALSE)

VarMotifs <- getVarDeviations(proj, name = "homerMatrix", plot = FALSE)

MotifScoreClusters <- getMarkerFeatures(
  proj,
  useMatrix    = "homerMatrix",
  groupBy      = "neoadjuvant",
  bias         = c("TSSEnrichment", "log10(nFrags)"),
  testMethod   = "wilcoxon",
  useSeqnames  = "z"
)

# ===== 取前 top_n_var_motifs 个变异度最高的 motif，并构建 Mean 矩阵 =====
fh <- function(x) hclust(dist(x), method = "ward.D2")  # 若后续仍想要聚类可用（本脚本里不再对行聚类）
idy <- which(mcols(MotifScoreClusters)$name %in% VarMotifs$name[seq_len(min(top_n_var_motifs, length(VarMotifs$name)))])
mtx2 <- assays(MotifScoreClusters[idy, ])$Mean %>%
  `rownames<-`(mcols(MotifScoreClusters)$name[idy])

# ===== 计算每个 motif 的行均值并进行筛选与排序 =====
row_means <- rowMeans(mtx2)

if (use_abs) {
  keep_idx <- which(abs(row_means) > threshold)
} else {
  keep_idx <- which(row_means > threshold)
}

if (length(keep_idx) == 0) {
  stop(sprintf("No motifs passed the threshold (threshold=%s, use_abs=%s). Try lowering the threshold.", threshold, use_abs))
}

mtx2_filt <- mtx2[keep_idx, , drop = FALSE]
row_means_filt <- row_means[keep_idx]

ord <- order(row_means_filt, decreasing = decreasing_order)
mtx2_filt <- mtx2_filt[ord, , drop = FALSE]

# ===== 颜色映射与绘图（保持行顺序，不进行行聚类）=====
col_fun2 <- colorRamp2(
  c(-5, -2.5, 0, 2, 5),
  paletteContinuous(set = "solarExtra", n = 5)
)

ht2 <- Heatmap(
  mtx2_filt,
  name = "Mean motif score",
  cluster_rows = FALSE,           # 保持你刚刚的排序，不再按层次聚类改变行顺序
  cluster_columns = FALSE,
  show_row_dend = FALSE,
  col = col_fun2
)

options(repr.plot.width = 10, repr.plot.height = 10)
p6 <- ComplexHeatmap::draw(ht2)

# ===== 导出 PDF =====
plotPDF(
  p6,
  name = "epi-Motif-Heatmap-filtered-sorted",  # 新文件名，避免覆盖原图
  width = 6,
  height = 8,
  ArchRProj = proj,
  addDOC = FALSE
)

# ===== 可选：打印一些摘要信息 =====
message(sprintf("Motifs input: %d | Passed filter: %d | Threshold: %s | use_abs=%s",
                nrow(mtx2), nrow(mtx2_filt), threshold, use_abs))


In [ ]:
markerPeaks <- getMarkerFeatures(
    ArchRProj = proj,
    useMatrix = "PeakMatrix", 
    groupBy = "neoadjuvant",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
markerTest <- getMarkerFeatures(
  ArchRProj = proj, 
  useMatrix = "PeakMatrix",
  groupBy = "neoadjuvant",
  testMethod = "wilcoxon",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  useGroups = "Post-NAT",
  bgdGroups = "Non-NAT"
)
pma <- plotMarkers(seMarker = markerTest, name = "Post-NAT", cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1", plotAs = "MA")
plotPDF(pma, name = "epi-Markers-MA-Volcano", width = 5, height = 5, ArchRProj = proj, addDOC = FALSE)

In [ ]:
seGroupMotif <- getGroupSE(ArchRProj = proj, useMatrix = "MotifMatrix", groupBy = "neoadjuvant")
seZ <- seGroupMotif[rowData(seGroupMotif)$seqnames=="z",]
rowData(seZ)$maxDelta <- lapply(seq_len(ncol(seZ)), function(x){
  rowMaxs(assay(seZ) - assay(seZ)[,x])
}) %>% Reduce("cbind", .) %>% rowMaxs
corGIM_MM <- correlateMatrices(
    ArchRProj = proj,
    useMatrix1 = "GeneIntegrationMatrix",
    useMatrix2 = "MotifMatrix",
    reducedDims = "IterativeLSI"
)
corGIM_MM$maxDelta <- rowData(seZ)[match(corGIM_MM$MotifMatrix_name, rowData(seZ)$name), "maxDelta"]
corGIM_MM <- corGIM_MM[order(abs(corGIM_MM$cor), decreasing = TRUE), ]
corGIM_MM <- corGIM_MM[which(!duplicated(gsub("\\-.*","",corGIM_MM[,"MotifMatrix_name"]))), ]
corGIM_MM$TFRegulator <- "NO"
corGIM_MM$TFRegulator[which(corGIM_MM$cor > 0.5 & corGIM_MM$padj < 0.01 & corGIM_MM$maxDelta > quantile(corGIM_MM$maxDelta, 0.75))] <- "YES"
sort(corGIM_MM[corGIM_MM$TFRegulator=="YES",1])

corGIM_MM$TF <- gsub("\\-.*","", corGIM_MM$MotifMatrix_name)
lab_dat <- subset(corGIM_MM, TFRegulator == "YES")

p <- ggplot(corGIM_MM, aes(cor, maxDelta, color = TFRegulator)) +
  geom_point() +
  theme_ArchR() +
  geom_vline(xintercept = 0, lty = "dashed") +
  scale_color_manual(values = c("NO"="darkgrey", "YES"="firebrick3")) +
  xlab("Correlation To Gene Expression") +
  ylab("Max TF Motif Delta") +
  scale_y_continuous(expand = c(0,0), limits = c(0, max(corGIM_MM$maxDelta)*1.05)) +
  ggrepel::geom_text_repel(
    data = lab_dat,
    aes(label = TF),
    show.legend = FALSE,
    size = 3,
    box.padding = 0.3,
    point.padding = 0.1,
    segment.size = 0.2,
    max.overlaps = Inf
  )

p

ggsave("/data/hanxue/figures/epi-TF-Correlation.svg",plot = p, width = 6, height = 6, device = "svg")

In [ ]:
motifPositions <- getPositions(proj)
motifPositions

In [ ]:
## ArchR分析高变TF的结果
# tfs <- c(
#   'BCL11A','CEBPB','ELF5','ESR1','ETV7','FOSL1','FOXA1',
#   'FOXO6','GATA3','HIVEP1','HLF','MAFF','MAFK','MESP2',
#   'NFE2L2','NFKB2','NR2F6','PRDM1','RARA','RELB','RUNX3',
#   'SOX9','STAT3','STAT5A','TCF7','TCF7L2','TEAD1','TEAD3',
#   'TP63','TP73','ZSCAN16')


## DSG1 WIF1 ECRG4 KRT14 DSG3 NPY1R AGTR1 RHOXF1P3 FGFR3 KRT8 VIM 对应的TF
tfs <- c(
  "TP63", "KLF4", "GRHL1", "GRHL3",
  "TP53", "TCF7L2", "LEF1", "SP1",
  "NFKB1", "RELA",
  "KLF5", "FOXN1",
  "JUN", "FOS",
  "CREB1", "STAT3",
  "HIF1A", "NFATC1", "STAT1",
  "RHOX family", "HOX TFs",
  "ELK1", "ETS1",
  "HNF4A", "GATA6", "FOXA1", "FOXA2",
  "SNAI1", "TWIST1", "ZEB1", "ZEB2"
)
markerMotifs <- unique(unlist(lapply(tfs, function(x)
  grep(x, names(motifPositions), value = TRUE, ignore.case = TRUE)
)))
markerMotifs

In [ ]:
if(is.null(proj@projectMetadata$GroupCoverages$neoadjuvant)){
  proj <- addGroupCoverages(ArchRProj = proj, groupBy = "neoadjuvant")
}

In [ ]:
seFoot <- getFootprints(
  ArchRProj = proj, 
  positions = motifPositions[markerMotifs], 
  groupBy = "neoadjuvant"
)

In [ ]:
plotFootprints(
  seFoot = seFoot,
  ArchRProj = proj, 
  normMethod = "Subtract",
  plotName = "Epi-Footprints-Subtract-Bias-RNA-DEG",
  addDOC = FALSE,
  smoothWindow = 5
)

In [ ]:
markerGenes  <- c(
    "DSG1", "WIF1", "ECRG4", "KRT14", "DSG3", "NPY1R", "AGTR1", "RHOXF1P3", "FGFR3", "KRT8", "VIM"
  )

p <- plotBrowserTrack(
    ArchRProj = proj, 
    groupBy = "neoadjuvant", 
    geneSymbol = markerGenes, 
    upstream = 50000,
    downstream = 50000
)

In [ ]:
plotPDF(plotList = p, 
    name = "Epi-Tracks-Marker-Genes", 
    ArchRProj = proj, 
    addDOC = FALSE, width = 5, height = 5)

#### epi-After

In [6]:
neo_after <- getCellNames(ArchRProj = proj)[proj$Sample %in% c("11T", "12T", "13T", "19T")]
proj_after <- proj[neo_after,]
proj_after$effect <- ifelse(
  proj_after$Sample %in% c("13T", "19T"),
  "better",
  "worse"
)

In [ ]:
VarMotifs <- getVarDeviations(proj_after, name = "homerMatrix", plot = F)
MotifScoreClusters <- getMarkerFeatures(proj_after, 
                                        useMatrix = "homerMatrix", 
                                        groupBy = "effect",
                                        bias = c("TSSEnrichment", "log10(nFrags)"),
                                        testMethod = "wilcoxon", 
                                        useSeqnames = "z")

fh <- function(x) hclust(dist(x), method="ward.D2")
idy <- which(mcols(MotifScoreClusters)$name %in% VarMotifs$name[c(1:50)])
row_means <- rowMeans(mtx2)   # 每个motif的平均score
mtx2 <- mtx2[order(row_means, decreasing = TRUE), ]   # 从大到小排序
mtx2 <- assays(MotifScoreClusters[idy,])$Mean %>% `rownames<-`(., mcols(MotifScoreClusters)$name[idy])

col_fun2 <- colorRamp2(c(-5,-2.5,0,2,5), paletteContinuous(set = "solarExtra", n = 5))
ht2 <- Heatmap(mtx2, name = "Mean motif score", cluster_rows = fh, cluster_columns = F, show_row_dend = F, col = col_fun2)
options(repr.plot.width=10, repr.plot.height=10)
p6 <- draw(ht2)
plotPDF(p6, name = "epi-After-Motif-Heatmap", width = 6, height = 8, ArchRProj = proj_after, addDOC = FALSE)

In [ ]:
markerPeaks <- getMarkerFeatures(
    ArchRProj = proj_after,
    useMatrix = "PeakMatrix", 
    groupBy = "effect",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
markerTest <- getMarkerFeatures(
  ArchRProj = proj_after, 
  useMatrix = "PeakMatrix",
  groupBy = "effect",
  testMethod = "wilcoxon",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  useGroups = "better",
  bgdGroups = "worse"
)
pma <- plotMarkers(seMarker = markerTest, name = "better", cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1", plotAs = "MA")
plotPDF(pma, name = "epi-After-Markers-MA-Volcano", width = 5, height = 5, ArchRProj = proj_after, addDOC = FALSE)

In [ ]:
seGroupMotif <- getGroupSE(ArchRProj = proj_after, useMatrix = "MotifMatrix", groupBy = "effect")
seZ <- seGroupMotif[rowData(seGroupMotif)$seqnames=="z",]
rowData(seZ)$maxDelta <- lapply(seq_len(ncol(seZ)), function(x){
  rowMaxs(assay(seZ) - assay(seZ)[,x])
}) %>% Reduce("cbind", .) %>% rowMaxs
corGIM_MM <- correlateMatrices(
    ArchRProj = proj_after,
    useMatrix1 = "GeneIntegrationMatrix",
    useMatrix2 = "MotifMatrix",
    reducedDims = "IterativeLSI"
)
corGIM_MM$maxDelta <- rowData(seZ)[match(corGIM_MM$MotifMatrix_name, rowData(seZ)$name), "maxDelta"]
corGIM_MM <- corGIM_MM[order(abs(corGIM_MM$cor), decreasing = TRUE), ]
corGIM_MM <- corGIM_MM[which(!duplicated(gsub("\\-.*","",corGIM_MM[,"MotifMatrix_name"]))), ]
corGIM_MM$TFRegulator <- "NO"
corGIM_MM$TFRegulator[which(corGIM_MM$cor > 0.5 & corGIM_MM$padj < 0.01 & corGIM_MM$maxDelta > quantile(corGIM_MM$maxDelta, 0.75))] <- "YES"
sort(corGIM_MM[corGIM_MM$TFRegulator=="YES",1])

corGIM_MM$TF <- gsub("\\-.*","", corGIM_MM$MotifMatrix_name)
lab_dat <- subset(corGIM_MM, TFRegulator == "YES")

p <- ggplot(corGIM_MM, aes(cor, maxDelta, color = TFRegulator)) +
  geom_point() +
  theme_ArchR() +
  geom_vline(xintercept = 0, lty = "dashed") +
  scale_color_manual(values = c("NO"="darkgrey", "YES"="firebrick3")) +
  xlab("Correlation To Gene Expression") +
  ylab("Max TF Motif Delta") +
  scale_y_continuous(expand = c(0,0), limits = c(0, max(corGIM_MM$maxDelta)*1.05)) +
  ggrepel::geom_text_repel(
    data = lab_dat,
    aes(label = TF),
    show.legend = FALSE,
    size = 3,
    box.padding = 0.3,
    point.padding = 0.1,
    segment.size = 0.2,
    max.overlaps = Inf
  )

p

ggsave("/data/hanxue/figures/epi-After-TF-Correlation.svg",plot = p, width = 6, height = 6, device = "svg")

In [ ]:
motifPositions <- getPositions(proj_after)
motifPositions
# tfs <- c(
#   'AR','ATF3','BATF','CEBPB','E4F1','EGR1','EGR2','EGR3','ESR1','FOSL1','FOSL2',
#   'FOXA1','HNF4G','HOXC13','JDP2','MAFF','MAFK','MEF2D','MLX','NFKB2','NR6A1',
#   'RUNX3','SMARCC1','SOX15','SOX9','TCF7','TCF7L2','TEAD1','TP63','TP73','USF2','ZSCAN16'
# )

tfs <- c(
  "SNAI1", "SNAI2", "TWIST1", "ZEB1", "ZEB2", "NFKB1", "RELA", "JUN", "FOS", # VIM
  "HNF4A", "FOXA1", "FOXA2", "GATA6", "CEBPB", # KRT8
  "TP63", "KLF5", "FOXN1", "GRHL1" # KRT14
)


markerMotifs <- unique(unlist(lapply(tfs, function(x)
  grep(x, names(motifPositions), value = TRUE, ignore.case = TRUE)
)))
markerMotifs

In [ ]:
if(is.null(proj_after@projectMetadata$GroupCoverages$effect)){
  proj_after <- addGroupCoverages(ArchRProj = proj_after, groupBy = "effect")
}
seFoot <- getFootprints(
  ArchRProj = proj_after, 
  positions = motifPositions[markerMotifs], 
  groupBy = "effect"
)
plotFootprints(
  seFoot = seFoot,
  ArchRProj = proj_after, 
  normMethod = "Subtract",
  plotName = "Epi-Effect-Footprints-Subtract-Bias-RNA-DEG",
  addDOC = FALSE,
  smoothWindow = 5
)

In [7]:
markerGenes  <- c(
     "KRT14", "KRT8", "VIM"
  )

p <- plotBrowserTrack(
    ArchRProj = proj_after, 
    groupBy = "effect", 
    geneSymbol = markerGenes, 
    upstream = 50000,
    downstream = 50000
)

ArchR logging to : ArchRLogs/ArchR-plotBrowserTrack-29e8902023a4b2-Date-2025-09-25_Time-13-16-13.351934.log
If there is an issue, please report to github with logFile!

2025-09-25 13:16:13.532517 : Validating Region, 0.003 mins elapsed.



GRanges object with 3 ranges and 2 metadata columns:
      seqnames            ranges strand |     gene_id      symbol
         <Rle>         <IRanges>  <Rle> | <character> <character>
  [1]    chr17 41582279-41586921      - |        3861       KRT14
  [2]    chr12 52897187-52949954      - |        3856        KRT8
  [3]    chr10 17228259-17237593      + |        7431         VIM
  -------
  seqinfo: 24 sequences from hg38 genome


2025-09-25 13:16:13.643585 : Adding Bulk Tracks (1 of 3), 0.005 mins elapsed.

2025-09-25 13:16:26.091907 : Adding Feature Tracks (1 of 3), 0.212 mins elapsed.

2025-09-25 13:16:26.194567 : Adding Gene Tracks (1 of 3), 0.214 mins elapsed.

2025-09-25 13:16:26.666356 : Plotting, 0.222 mins elapsed.

2025-09-25 13:16:27.56746 : Adding Bulk Tracks (2 of 3), 0.237 mins elapsed.

2025-09-25 13:16:30.673233 : Adding Feature Tracks (2 of 3), 0.289 mins elapsed.

2025-09-25 13:16:30.735755 : Adding Gene Tracks (2 of 3), 0.29 mins elapsed.

2025-09-25 13:16:30.999286 : Plotting, 0.294 mins elapsed.

2025-09-25 13:16:31.526793 : Adding Bulk Tracks (3 of 3), 0.303 mins elapsed.

2025-09-25 13:16:34.766556 : Adding Feature Tracks (3 of 3), 0.357 mins elapsed.

2025-09-25 13:16:34.831178 : Adding Gene Tracks (3 of 3), 0.358 mins elapsed.

2025-09-25 13:16:35.108129 : Plotting, 0.363 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-plotBrowserTrack-29e8902023a4b2-Date-2025-09-25_Time-13-

In [8]:
plotPDF(p, name = "epi-After-Track", width = 5, height = 5, ArchRProj = proj_after, addDOC = FALSE)

Plotting Gtable!



NULL


Plotting Gtable!



NULL


Plotting Gtable!



NULL


#### T细胞

In [ ]:
proj <- loadArchRProject("./ArchRProject_T", showLogo = FALSE)

In [ ]:
proj$neoadjuvant <- ifelse(
  proj$neoadjuvant == "after_neoadjuvant", 
  "Post-NAT", 
  "Non-NAT"
)

neo_after <- getCellNames(ArchRProj = proj)[proj$Sample %ni% c("12N", "10N")]
proj <- proj[neo_after,]

In [ ]:
VarMotifs <- getVarDeviations(proj, name = "homerMatrix", plot = F)
MotifScoreClusters <- getMarkerFeatures(proj, 
                                        useMatrix = "homerMatrix", 
                                        groupBy = "neoadjuvant",
                                        bias = c("TSSEnrichment", "log10(nFrags)"),
                                        testMethod = "wilcoxon", 
                                        useSeqnames = "z")

fh <- function(x) hclust(dist(x), method="ward.D2")
idy <- which(mcols(MotifScoreClusters)$name %in% VarMotifs$name[c(1:50)])
mtx2 <- assays(MotifScoreClusters[idy,])$Mean %>% `rownames<-`(., mcols(MotifScoreClusters)$name[idy])

col_fun2 <- colorRamp2(c(-5,-2.5,0,2,5), paletteContinuous(set = "solarExtra", n = 5))
ht2 <- Heatmap(mtx2, name = "Mean motif score", cluster_rows = fh, cluster_columns = F, show_row_dend = F, col = col_fun2)
options(repr.plot.width=10, repr.plot.height=10)
p6 <- draw(ht2)
plotPDF(p6, name = "T-Motif-Heatmap", width = 6, height = 8, ArchRProj = proj, addDOC = FALSE)

In [ ]:
markerPeaks <- getMarkerFeatures(
    ArchRProj = proj,
    useMatrix = "PeakMatrix", 
    groupBy = "neoadjuvant",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
markerTest <- getMarkerFeatures(
  ArchRProj = proj, 
  useMatrix = "PeakMatrix",
  groupBy = "neoadjuvant",
  testMethod = "wilcoxon",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  useGroups = "Post-NAT",
  bgdGroups = "Non-NAT"
)
pma <- plotMarkers(seMarker = markerTest, name = "Post-NAT", cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1", plotAs = "MA")
plotPDF(pma, name = "T-Markers-MA-Volcano", width = 5, height = 5, ArchRProj = proj, addDOC = FALSE)

In [ ]:
seGroupMotif <- getGroupSE(ArchRProj = proj, useMatrix = "MotifMatrix", groupBy = "neoadjuvant")
seZ <- seGroupMotif[rowData(seGroupMotif)$seqnames=="z",]
rowData(seZ)$maxDelta <- lapply(seq_len(ncol(seZ)), function(x){
  rowMaxs(assay(seZ) - assay(seZ)[,x])
}) %>% Reduce("cbind", .) %>% rowMaxs
corGIM_MM <- correlateMatrices(
    ArchRProj = proj,
    useMatrix1 = "GeneIntegrationMatrix",
    useMatrix2 = "MotifMatrix",
    reducedDims = "IterativeLSI"
)
corGIM_MM$maxDelta <- rowData(seZ)[match(corGIM_MM$MotifMatrix_name, rowData(seZ)$name), "maxDelta"]
corGIM_MM <- corGIM_MM[order(abs(corGIM_MM$cor), decreasing = TRUE), ]
corGIM_MM <- corGIM_MM[which(!duplicated(gsub("\\-.*","",corGIM_MM[,"MotifMatrix_name"]))), ]
corGIM_MM$TFRegulator <- "NO"
corGIM_MM$TFRegulator[which(corGIM_MM$cor > 0.5 & corGIM_MM$padj < 0.01 & corGIM_MM$maxDelta > quantile(corGIM_MM$maxDelta, 0.75))] <- "YES"
sort(corGIM_MM[corGIM_MM$TFRegulator=="YES",1])

corGIM_MM$TF <- gsub("\\-.*","", corGIM_MM$MotifMatrix_name)
lab_dat <- subset(corGIM_MM, TFRegulator == "YES")

p <- ggplot(corGIM_MM, aes(cor, maxDelta, color = TFRegulator)) +
  geom_point() +
  theme_ArchR() +
  geom_vline(xintercept = 0, lty = "dashed") +
  scale_color_manual(values = c("NO"="darkgrey", "YES"="firebrick3")) +
  xlab("Correlation To Gene Expression") +
  ylab("Max TF Motif Delta") +
  scale_y_continuous(expand = c(0,0), limits = c(0, max(corGIM_MM$maxDelta)*1.05)) +
  ggrepel::geom_text_repel(
    data = lab_dat,
    aes(label = TF),
    show.legend = FALSE,
    size = 3,
    box.padding = 0.3,
    point.padding = 0.1,
    segment.size = 0.2,
    max.overlaps = Inf
  )

p

ggsave("/data/hanxue/figures/T-TF-Correlation.svg",plot = p, width = 6, height = 6, device = "svg")

In [ ]:
motifPositions <- getPositions(proj)
motifPositions

# tfs <- c(
#   'BATF', 'EOMES', 'MBD2', 'RUNX3','STAT2'
# )
## CXCL13 PDCD1 CD200
tfs <- c(
  "NFKB1", "RELA", "STAT3", "IRF4", "IRF5", "IRF8", "BATF", "BCL6",
  "NFATC1", "NFATC2", "FOXO1", "IRF9", "STAT4", "JUN", "FOS",
  "NR4A1", "NR4A2", "NR4A3",
  "SP1", "CREB1", "STAT6", "GATA3"
)
markerMotifs <- unique(unlist(lapply(tfs, function(x)
  grep(x, names(motifPositions), value = TRUE, ignore.case = TRUE)
)))
markerMotifs

if(is.null(proj@projectMetadata$GroupCoverages$neoadjuvant)){
  proj <- addGroupCoverages(ArchRProj = proj, groupBy = "neoadjuvant")
}

seFoot <- getFootprints(
  ArchRProj = proj, 
  positions = motifPositions[markerMotifs], 
  groupBy = "neoadjuvant"
)

plotFootprints(
  seFoot = seFoot,
  ArchRProj = proj, 
  normMethod = "Subtract",
  plotName = "T-Footprints-Subtract-Bias-RNA-DEG",
  addDOC = FALSE,
  smoothWindow = 5
)

In [ ]:
markerGenes  <- c(
    "CXCL13", 
    "PDCD1",
    "CD200"
  )

p <- plotBrowserTrack(
    ArchRProj = proj, 
    groupBy = "neoadjuvant", 
    geneSymbol = markerGenes, 
    upstream = 50000,
    downstream = 50000
)

In [ ]:
plotPDF(plotList = p, 
    name = "T-Tracks-Marker-Genes", 
    ArchRProj = proj, 
    addDOC = FALSE, width = 5, height = 5)

#### T after

In [ ]:
neo_after <- getCellNames(ArchRProj = proj)[proj$Sample %in% c("11T", "12T", "13T", "19T")]
proj_after <- proj[neo_after,]
proj_after$effect <- ifelse(
  proj_after$Sample %in% c("13T", "19T"),
  "better",
  "worse"
)

In [ ]:
VarMotifs <- getVarDeviations(proj_after, name = "homerMatrix", plot = F)
MotifScoreClusters <- getMarkerFeatures(proj_after, 
                                        useMatrix = "homerMatrix", 
                                        groupBy = "effect",
                                        bias = c("TSSEnrichment", "log10(nFrags)"),
                                        testMethod = "wilcoxon", 
                                        useSeqnames = "z")

fh <- function(x) hclust(dist(x), method="ward.D2")
idy <- which(mcols(MotifScoreClusters)$name %in% VarMotifs$name[c(1:50)])
row_means <- rowMeans(mtx2)   # 每个motif的平均score
mtx2 <- mtx2[order(row_means, decreasing = TRUE), ]   # 从大到小排序
mtx2 <- assays(MotifScoreClusters[idy,])$Mean %>% `rownames<-`(., mcols(MotifScoreClusters)$name[idy])

col_fun2 <- colorRamp2(c(-5,-2.5,0,2,5), paletteContinuous(set = "solarExtra", n = 5))
ht2 <- Heatmap(mtx2, name = "Mean motif score", cluster_rows = fh, cluster_columns = F, show_row_dend = F, col = col_fun2)
options(repr.plot.width=10, repr.plot.height=10)
p6 <- draw(ht2)
plotPDF(p6, name = "T-After-Motif-Heatmap", width = 6, height = 8, ArchRProj = proj_after, addDOC = FALSE)

In [ ]:
markerPeaks <- getMarkerFeatures(
    ArchRProj = proj_after,
    useMatrix = "PeakMatrix", 
    groupBy = "effect",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
markerTest <- getMarkerFeatures(
  ArchRProj = proj_after, 
  useMatrix = "PeakMatrix",
  groupBy = "effect",
  testMethod = "wilcoxon",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  useGroups = "better",
  bgdGroups = "worse"
)
pma <- plotMarkers(seMarker = markerTest, name = "better", cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1", plotAs = "MA")
plotPDF(pma, name = "T-After-Markers-MA-Volcano", width = 5, height = 5, ArchRProj = proj_after, addDOC = FALSE)

In [ ]:
seGroupMotif <- getGroupSE(ArchRProj = proj_after, useMatrix = "MotifMatrix", groupBy = "effect")
seZ <- seGroupMotif[rowData(seGroupMotif)$seqnames=="z",]
rowData(seZ)$maxDelta <- lapply(seq_len(ncol(seZ)), function(x){
  rowMaxs(assay(seZ) - assay(seZ)[,x])
}) %>% Reduce("cbind", .) %>% rowMaxs
corGIM_MM <- correlateMatrices(
    ArchRProj = proj_after,
    useMatrix1 = "GeneIntegrationMatrix",
    useMatrix2 = "MotifMatrix",
    reducedDims = "IterativeLSI"
)
corGIM_MM$maxDelta <- rowData(seZ)[match(corGIM_MM$MotifMatrix_name, rowData(seZ)$name), "maxDelta"]
corGIM_MM <- corGIM_MM[order(abs(corGIM_MM$cor), decreasing = TRUE), ]
corGIM_MM <- corGIM_MM[which(!duplicated(gsub("\\-.*","",corGIM_MM[,"MotifMatrix_name"]))), ]
corGIM_MM$TFRegulator <- "NO"
corGIM_MM$TFRegulator[which(corGIM_MM$cor > 0.5 & corGIM_MM$padj < 0.01 & corGIM_MM$maxDelta > quantile(corGIM_MM$maxDelta, 0.75))] <- "YES"
sort(corGIM_MM[corGIM_MM$TFRegulator=="YES",1])

corGIM_MM$TF <- gsub("\\-.*","", corGIM_MM$MotifMatrix_name)
lab_dat <- subset(corGIM_MM, TFRegulator == "YES")

p <- ggplot(corGIM_MM, aes(cor, maxDelta, color = TFRegulator)) +
  geom_point() +
  theme_ArchR() +
  geom_vline(xintercept = 0, lty = "dashed") +
  scale_color_manual(values = c("NO"="darkgrey", "YES"="firebrick3")) +
  xlab("Correlation To Gene Expression") +
  ylab("Max TF Motif Delta") +
  scale_y_continuous(expand = c(0,0), limits = c(0, max(corGIM_MM$maxDelta)*1.05)) +
  ggrepel::geom_text_repel(
    data = lab_dat,
    aes(label = TF),
    show.legend = FALSE,
    size = 3,
    box.padding = 0.3,
    point.padding = 0.1,
    segment.size = 0.2,
    max.overlaps = Inf
  )

p

ggsave("/data/hanxue/figures/T-After-TF-Correlation.svg",plot = p, width = 6, height = 6, device = "svg")

In [ ]:
motifPositions <- getPositions(proj_after)
motifPositions
tfs <- c(
 'FOXN2','FOXN3-AS1','IRF4','NR1D1','RUNX3','TBX19','TGIF2'
)
markerMotifs <- unique(unlist(lapply(tfs, function(x)
  grep(x, names(motifPositions), value = TRUE, ignore.case = TRUE)
)))
markerMotifs

In [ ]:
if(is.null(proj_after@projectMetadata$GroupCoverages$effect)){
  proj_after <- addGroupCoverages(ArchRProj = proj_after, groupBy = "effect")
}
seFoot <- getFootprints(
  ArchRProj = proj_after, 
  positions = motifPositions[markerMotifs], 
  groupBy = "effect"
)
plotFootprints(
  seFoot = seFoot,
  ArchRProj = proj_after, 
  normMethod = "Subtract",
  plotName = "T-Effect-Footprints-Subtract-Bias-ATAC-TF",
  addDOC = FALSE,
  smoothWindow = 5
)

In [ ]:
markerGenes  <- c(
    "CXCL13", 
    "PDCD1",
    "CD200"
  )

p <- plotBrowserTrack(
    ArchRProj = proj_after, 
    groupBy = "effect", 
    geneSymbol = markerGenes, 
    upstream = 50000,
    downstream = 50000
)

In [ ]:
plotPDF(plotList = p, 
    name = "T-After-Tracks-Marker-Genes", 
    ArchRProj = proj_after, 
    addDOC = FALSE, width = 5, height = 5)

#### 髓系细胞

In [9]:
proj <- loadArchRProject("./ArchRProject_Myeloid", showLogo = FALSE)

Successfully loaded ArchRProject!



In [10]:
proj$neoadjuvant <- ifelse(
  proj$neoadjuvant == "after_neoadjuvant", 
  "Post-NAT", 
  "Non-NAT"
)

neo_after <- getCellNames(ArchRProj = proj)[proj$Sample %ni% c("12N", "10N")]
proj <- proj[neo_after,]

Dropping ImputeWeights Since You Are Subsetting Cells! ImputeWeights is a cell-x-cell Matrix!



In [ ]:
VarMotifs <- getVarDeviations(proj, name = "homerMatrix", plot = F)
MotifScoreClusters <- getMarkerFeatures(proj, 
                                        useMatrix = "homerMatrix", 
                                        groupBy = "neoadjuvant",
                                        bias = c("TSSEnrichment", "log10(nFrags)"),
                                        testMethod = "wilcoxon", 
                                        useSeqnames = "z")

fh <- function(x) hclust(dist(x), method="ward.D2")
idy <- which(mcols(MotifScoreClusters)$name %in% VarMotifs$name[c(1:50)])
mtx2 <- assays(MotifScoreClusters[idy,])$Mean %>% `rownames<-`(., mcols(MotifScoreClusters)$name[idy])

col_fun2 <- colorRamp2(c(-5,-2.5,0,2,5), paletteContinuous(set = "solarExtra", n = 5))
ht2 <- Heatmap(mtx2, name = "Mean motif score", cluster_rows = fh, cluster_columns = F, show_row_dend = F, col = col_fun2)
options(repr.plot.width=10, repr.plot.height=10)
p6 <- draw(ht2)
plotPDF(p6, name = "Mye-Motif-Heatmap", width = 6, height = 8, ArchRProj = proj, addDOC = FALSE)

In [ ]:
markerPeaks <- getMarkerFeatures(
    ArchRProj = proj,
    useMatrix = "PeakMatrix", 
    groupBy = "neoadjuvant",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
markerTest <- getMarkerFeatures(
  ArchRProj = proj, 
  useMatrix = "PeakMatrix",
  groupBy = "neoadjuvant",
  testMethod = "wilcoxon",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  useGroups = "Post-NAT",
  bgdGroups = "Non-NAT"
)
pma <- plotMarkers(seMarker = markerTest, name = "Post-NAT", cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1", plotAs = "MA")
plotPDF(pma, name = "Mye-Markers-MA-Volcano", width = 5, height = 5, ArchRProj = proj, addDOC = FALSE)

In [ ]:
seGroupMotif <- getGroupSE(ArchRProj = proj, useMatrix = "MotifMatrix", groupBy = "neoadjuvant")
seZ <- seGroupMotif[rowData(seGroupMotif)$seqnames=="z",]
rowData(seZ)$maxDelta <- lapply(seq_len(ncol(seZ)), function(x){
  rowMaxs(assay(seZ) - assay(seZ)[,x])
}) %>% Reduce("cbind", .) %>% rowMaxs
corGIM_MM <- correlateMatrices(
    ArchRProj = proj,
    useMatrix1 = "GeneIntegrationMatrix",
    useMatrix2 = "MotifMatrix",
    reducedDims = "IterativeLSI"
)
corGIM_MM$maxDelta <- rowData(seZ)[match(corGIM_MM$MotifMatrix_name, rowData(seZ)$name), "maxDelta"]
corGIM_MM <- corGIM_MM[order(abs(corGIM_MM$cor), decreasing = TRUE), ]
corGIM_MM <- corGIM_MM[which(!duplicated(gsub("\\-.*","",corGIM_MM[,"MotifMatrix_name"]))), ]
corGIM_MM$TFRegulator <- "NO"
corGIM_MM$TFRegulator[which(corGIM_MM$cor > 0.5 & corGIM_MM$padj < 0.01 & corGIM_MM$maxDelta > quantile(corGIM_MM$maxDelta, 0.75))] <- "YES"
sort(corGIM_MM[corGIM_MM$TFRegulator=="YES",1])

corGIM_MM$TF <- gsub("\\-.*","", corGIM_MM$MotifMatrix_name)
lab_dat <- subset(corGIM_MM, TFRegulator == "YES")

p <- ggplot(corGIM_MM, aes(cor, maxDelta, color = TFRegulator)) +
  geom_point() +
  theme_ArchR() +
  geom_vline(xintercept = 0, lty = "dashed") +
  scale_color_manual(values = c("NO"="darkgrey", "YES"="firebrick3")) +
  xlab("Correlation To Gene Expression") +
  ylab("Max TF Motif Delta") +
  scale_y_continuous(expand = c(0,0), limits = c(0, max(corGIM_MM$maxDelta)*1.05)) +
  ggrepel::geom_text_repel(
    data = lab_dat,
    aes(label = TF),
    show.legend = FALSE,
    size = 3,
    box.padding = 0.3,
    point.padding = 0.1,
    segment.size = 0.2,
    max.overlaps = Inf
  )

p

ggsave("/data/hanxue/figures/Mye-TF-Correlation.svg",plot = p, width = 6, height = 6, device = "svg")

In [13]:
motifPositions <- getPositions(proj)
motifPositions

tfs <- c(
  'EPAS1','FOSL1','HIVEP2','IRF5','MAFB','NFKB1','PAX8','REL','RFX3','STAT2','TCF21','TEAD4','TFEC','ZEB1'
)
## SPP1 CXCL9 FOLR2 C3 MMP9
# tfs <-  c(
#   "NFKB1", "RELA", "STAT3", "RUNX2", "HIF1A", "JUN", "FOS", "SP1",
#   "STAT1", "IRF1", "IRF8",
#   "ETS1",
#   "PPARG", "STAT6", "CEBPB", "MafB",
#   "CEBPA", "HNF1A", "HNF4A"
# )
markerMotifs <- unique(unlist(lapply(tfs, function(x)
  grep(x, names(motifPositions), value = TRUE, ignore.case = TRUE)
)))
markerMotifs

# if(is.null(proj@projectMetadata$GroupCoverages$neoadjuvant)){
#   proj <- addGroupCoverages(ArchRProj = proj, groupBy = "neoadjuvant")
# }

# seFoot <- getFootprints(
#   ArchRProj = proj, 
#   positions = motifPositions[markerMotifs], 
#   groupBy = "neoadjuvant"
# )

# plotFootprints(
#   seFoot = seFoot,
#   ArchRProj = proj, 
#   normMethod = "Subtract",
#   plotName = "Mye-Footprints-Subtract-Bias-ATAC-TF",
#   addDOC = FALSE,
#   smoothWindow = 5
# )

GRangesList object of length 332:
$AP.1.bZIP_1
GRanges object with 67738 ranges and 1 metadata column:
          seqnames              ranges strand |     score
             <Rle>           <IRanges>  <Rle> | <numeric>
      [1]     chr1       832379-832388      + |   9.05348
      [2]     chr1       967006-967015      - |   9.04335
      [3]     chr1     1006754-1006763      - |   9.55856
      [4]     chr1     1022065-1022074      + |   8.77324
      [5]     chr1     1022064-1022073      - |   9.14812
      ...      ...                 ...    ... .       ...
  [67734]     chrX 155334754-155334763      + |   9.64631
  [67735]     chrX 155334560-155334569      - |   9.04335
  [67736]     chrX 155334753-155334762      - |   9.55022
  [67737]     chrX 155888402-155888411      + |   9.77651
  [67738]     chrX 155888401-155888410      - |   9.40673
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

...
<331 more elements>

[1] "Pax8.Paired.Homeobox_216" "NFkB.p65.Rel.RHD_180"    
[3] "Tcf21.bHLH_287"           "TEAD4.TEA_293"

In [ ]:
markerGenes  <- c(
    "SPP1", 
    "CXCL9",
    "MMP9"
  )

p <- plotBrowserTrack(
    ArchRProj = proj, 
    groupBy = "neoadjuvant", 
    geneSymbol = markerGenes, 
    upstream = 50000,
    downstream = 50000
)

In [ ]:
plotPDF(plotList = p, 
    name = "Mye-Tracks-Marker-Genes", 
    ArchRProj = proj, 
    addDOC = FALSE, width = 5, height = 5)

#### Mye After

In [ ]:
neo_after <- getCellNames(ArchRProj = proj)[proj$Sample %in% c("11T", "12T", "13T", "19T")]
proj_after <- proj[neo_after,]
proj_after$effect <- ifelse(
  proj_after$Sample %in% c("13T", "19T"),
  "better",
  "worse"
)

In [ ]:
VarMotifs <- getVarDeviations(proj_after, name = "homerMatrix", plot = F)
MotifScoreClusters <- getMarkerFeatures(proj_after, 
                                        useMatrix = "homerMatrix", 
                                        groupBy = "effect",
                                        bias = c("TSSEnrichment", "log10(nFrags)"),
                                        testMethod = "wilcoxon", 
                                        useSeqnames = "z")

fh <- function(x) hclust(dist(x), method="ward.D2")
idy <- which(mcols(MotifScoreClusters)$name %in% VarMotifs$name[c(1:50)])
row_means <- rowMeans(mtx2)   # 每个motif的平均score
mtx2 <- mtx2[order(row_means, decreasing = TRUE), ]   # 从大到小排序
mtx2 <- assays(MotifScoreClusters[idy,])$Mean %>% `rownames<-`(., mcols(MotifScoreClusters)$name[idy])

col_fun2 <- colorRamp2(c(-5,-2.5,0,2,5), paletteContinuous(set = "solarExtra", n = 5))
ht2 <- Heatmap(mtx2, name = "Mean motif score", cluster_rows = fh, cluster_columns = F, show_row_dend = F, col = col_fun2)
options(repr.plot.width=10, repr.plot.height=10)
p6 <- draw(ht2)
plotPDF(p6, name = "Mye-After-Motif-Heatmap", width = 6, height = 8, ArchRProj = proj_after, addDOC = FALSE)

In [ ]:
markerPeaks <- getMarkerFeatures(
    ArchRProj = proj_after,
    useMatrix = "PeakMatrix", 
    groupBy = "effect",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
markerTest <- getMarkerFeatures(
  ArchRProj = proj_after, 
  useMatrix = "PeakMatrix",
  groupBy = "effect",
  testMethod = "wilcoxon",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  useGroups = "better",
  bgdGroups = "worse"
)
pma <- plotMarkers(seMarker = markerTest, name = "better", cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1", plotAs = "MA")
plotPDF(pma, name = "Mye-After-Markers-MA-Volcano", width = 5, height = 5, ArchRProj = proj_after, addDOC = FALSE)

In [ ]:
seGroupMotif <- getGroupSE(ArchRProj = proj_after, useMatrix = "MotifMatrix", groupBy = "effect")
seZ <- seGroupMotif[rowData(seGroupMotif)$seqnames=="z",]
rowData(seZ)$maxDelta <- lapply(seq_len(ncol(seZ)), function(x){
  rowMaxs(assay(seZ) - assay(seZ)[,x])
}) %>% Reduce("cbind", .) %>% rowMaxs
corGIM_MM <- correlateMatrices(
    ArchRProj = proj_after,
    useMatrix1 = "GeneIntegrationMatrix",
    useMatrix2 = "MotifMatrix",
    reducedDims = "IterativeLSI"
)
corGIM_MM$maxDelta <- rowData(seZ)[match(corGIM_MM$MotifMatrix_name, rowData(seZ)$name), "maxDelta"]
corGIM_MM <- corGIM_MM[order(abs(corGIM_MM$cor), decreasing = TRUE), ]
corGIM_MM <- corGIM_MM[which(!duplicated(gsub("\\-.*","",corGIM_MM[,"MotifMatrix_name"]))), ]
corGIM_MM$TFRegulator <- "NO"
corGIM_MM$TFRegulator[which(corGIM_MM$cor > 0.5 & corGIM_MM$padj < 0.01 & corGIM_MM$maxDelta > quantile(corGIM_MM$maxDelta, 0.75))] <- "YES"
sort(corGIM_MM[corGIM_MM$TFRegulator=="YES",1])

corGIM_MM$TF <- gsub("\\-.*","", corGIM_MM$MotifMatrix_name)
lab_dat <- subset(corGIM_MM, TFRegulator == "YES")

p <- ggplot(corGIM_MM, aes(cor, maxDelta, color = TFRegulator)) +
  geom_point() +
  theme_ArchR() +
  geom_vline(xintercept = 0, lty = "dashed") +
  scale_color_manual(values = c("NO"="darkgrey", "YES"="firebrick3")) +
  xlab("Correlation To Gene Expression") +
  ylab("Max TF Motif Delta") +
  scale_y_continuous(expand = c(0,0), limits = c(0, max(corGIM_MM$maxDelta)*1.05)) +
  ggrepel::geom_text_repel(
    data = lab_dat,
    aes(label = TF),
    show.legend = FALSE,
    size = 3,
    box.padding = 0.3,
    point.padding = 0.1,
    segment.size = 0.2,
    max.overlaps = Inf
  )

p

ggsave("/data/hanxue/figures/Mye-After-TF-Correlation.svg",plot = p, width = 6, height = 6, device = "svg")

In [ ]:
motifPositions <- getPositions(proj_after)
motifPositions
tfs <- c(
 'AR','ATF4','CEBPB','CEBPE','ETV3','FOSL1','FOXJ2','FOXN2','GRHL1','MAF','MITF','NFATC4','NFKB2','PRDM1','TEAD1','TFAP2A','USF2','ZEB1'
)
markerMotifs <- unique(unlist(lapply(tfs, function(x)
  grep(x, names(motifPositions), value = TRUE, ignore.case = TRUE)
)))
markerMotifs

if(is.null(proj_after@projectMetadata$GroupCoverages$effect)){
  proj_after <- addGroupCoverages(ArchRProj = proj_after, groupBy = "effect")
}
seFoot <- getFootprints(
  ArchRProj = proj_after, 
  positions = motifPositions[markerMotifs], 
  groupBy = "effect"
)
plotFootprints(
  seFoot = seFoot,
  ArchRProj = proj_after, 
  normMethod = "Subtract",
  plotName = "Mye-Effect-Footprints-Subtract-Bias-ATAC-TF",
  addDOC = FALSE,
  smoothWindow = 5
)

In [ ]:
markerGenes  <- c(
    "SPP1", 
    "CXCL9",
    "MMP9"
  )

p <- plotBrowserTrack(
    ArchRProj = proj_after, 
    groupBy = "effect", 
    geneSymbol = markerGenes, 
    upstream = 50000,
    downstream = 50000
)

In [ ]:
plotPDF(plotList = p, 
    name = "Mye-Effect-Tracks-Marker-Genes", 
    ArchRProj = proj, 
    addDOC = FALSE, width = 5, height = 5)

In [21]:
proj <- loadArchRProject("./ArchRProject", showLogo = FALSE)

Successfully loaded ArchRProject!



In [20]:
proj$majorType

[1] "Fibroblast"   "Endothelial"  "Epithelial"   "Endothelial" 
    [5] "Epithelial"   "Epithelial"   "Epithelial"   "Endothelial" 
    [9] "T cells"      "Epithelial"   "Epithelial"   "Myeloid"     
   [13] "T cells"      "Fibroblast"   "Epithelial"   "Endothelial" 
   [17] "Epithelial"   "PeriVascular" "Epithelial"   "Fibroblast"  
   [21] "Endothelial"  "Epithelial"   "Fibroblast"   "Epithelial"  
   [25] "Epithelial"   "Epithelial"   "PeriVascular" "Epithelial"  
   [29] "Epithelial"   "Mast"         "Fibroblast"   "Epithelial"  
   [33] "Fibroblast"   "Fibroblast"   "Epithelial"   "Endothelial" 
   [37] "Fibroblast"   "Epithelial"   "Epithelial"   "Epithelial"  
   [41] "Epithelial"   "Epithelial"   "Epithelial"   "Epithelial"  
   [45] "Epithelial"   "Epithelial"   "Epithelial"   "Epithelial"  
   [49] "Epithelial"   "T cells"      "Epithelial"   "Endothelial" 
   [53] "Endothelial"  "Endothelial"  "T cells"      "Endothelial" 
   [57] "Endothelial"  "PeriVascular" "Endothelial"  "T cells"     
   [61] "Epithelial"   "T cells"      "Endothelial"  "Epithelial"  
   [65] "Fibroblast"   "Epithelial"   "Epithelial"   "Epithelial"  
   [69] "Endothelial"  "Mast"         "Fibroblast"   "Endothelial" 
   [73] "Epithelial"   "Epithelial"   "Fibroblast"   "Fibroblast"  
   [77] "Myeloid"      "Fibroblast"   "Epithelial"   "Endothelial" 
   [81] "Endothelial"  "Fibroblast"   "Epithelial"   "Epithelial"  
   [85] "Fibroblast"   "Endothelial"  "Epithelial"   "PeriVascular"
   [89] "Epithelial"   "Epithelial"   "Epithelial"   "Fibroblast"  
   [93] "Fibroblast"   "PeriVascular" "Epithelial"   "Fibroblast"  
   [97] "Epithelial"   "Fibroblast"   "Fibroblast"   "Fibroblast"  
  [101] "Epithelial"   "Epithelial"   "Epithelial"   "Epithelial"  
  [105] "Epithelial"   "Epithelial"   "Mast"         "PeriVascular"
  [109] "Endothelial"  "Epithelial"   "Endothelial"  "Epithelial"  
  [113] "Myeloid"      "Endothelial"  "Fibroblast"   "Fibroblast"  
  [117] "Endothelial"  "PeriVascular" "Fibroblast"   "Endothelial" 
  [121] "Endothelial"  "Endothelial"  "Mast"         "Myeloid"     
  [125] "PeriVascular" "Fibroblast"   "Mast"         "Epithelial"  
  [129] "Endothelial"  "Fibroblast"   "PeriVascular" "Endothelial" 
  [133] "PeriVascular" "Endothelial"  "Myeloid"      "Fibroblast"  
  [137] "Endothelial"  "PeriVascular" "Fibroblast"   "Endothelial" 
  [141] "Epithelial"   "Epithelial"   "Epithelial"   "Epithelial"  
  [145] "Endothelial"  "T cells"      "PeriVascular" "Epithelial"  
  [149] "Endothelial"  "Myeloid"      "Fibroblast"   "Epithelial"  
  [153] "Fibroblast"   "Fibroblast"   "Fibroblast"   "Fibroblast"  
  [157] "Epithelial"   "Epithelial"   "T cells"      "Fibroblast"  
  [161] "Epithelial"   "Fibroblast"   "Epithelial"   "Fibroblast"  
  [165] "Fibroblast"   "Fibroblast"   "Epithelial"   "Epithelial"  
  [169] "Fibroblast"   "Epithelial"   "Endothelial"  "Endothelial" 
  [173] "Epithelial"   "Fibroblast"   "PeriVascular" "Epithelial"  
  [177] "Epithelial"   "Fibroblast"   "Fibroblast"   "Epithelial"  
  [181] "Fibroblast"   "Endothelial"  "Epithelial"   "Fibroblast"  
  [185] "Epithelial"   "Fibroblast"   "Epithelial"   "Myeloid"     
  [189] "Epithelial"   "Epithelial"   "PeriVascular" "Epithelial"  
  [193] "T cells"      "Fibroblast"   "Fibroblast"   "Epithelial"  
  [197] "Epithelial"   "Endothelial"  "Epithelial"   "Epithelial"  
  [201] "Endothelial"  "Epithelial"   "Epithelial"   "T cells"     
  [205] "Epithelial"   "Epithelial"   "Fibroblast"   "Fibroblast"  
  [209] "Epithelial"   "Epithelial"   "Endothelial"  "Endothelial" 
  [213] "Epithelial"   "Fibroblast"   "Epithelial"   "PeriVascular"
  [217] "Epithelial"   "PeriVascular" "Epithelial"   "Endothelial" 
  [221] "Epithelial"   "Endothelial"  "Epithelial"   "Epithelial"  
  [225] "Endothelial"  "Fibroblast"   "Endothelial"  "Endothelial" 
  [229] "Mast"         "Epithelial"   "Epithelial"   "Epithelial"  
  [233] "Fibroblast"   "Myeloid"      "Endothelial"  "Fibrob

In [22]:
proj$neoadjuvant <- ifelse(
  proj$neoadjuvant == "after_neoadjuvant", 
  "Post-NAT", 
  "Non-NAT"
)

In [23]:
neo_after <- getCellNames(ArchRProj = proj)[proj$Sample %ni% c("12N", "10N")]
proj <- proj[neo_after,]

Dropping ImputeWeights Since You Are Subsetting Cells! ImputeWeights is a cell-x-cell Matrix!



In [24]:
neo_after <- getCellNames(ArchRProj = proj)[proj$majorType %in% c("Epithelial", "T cells", "Myeloid")]
proj <- proj[neo_after,]

In [ ]:
# neo_after <- getCellNames(ArchRProj = proj)[proj$Sample %in% c("11T", "12T", "13T", "19T")]
# proj_after <- proj[neo_after,]

In [25]:
markerGenes  <- c(
    "VEGFA", "NRP2", 
    "HBEGF", "EGFR",
    "TNFSF10", "TNFRSF10B"
  )

p <- plotBrowserTrack(
    ArchRProj = proj, 
    groupBy = "neoadjuvant", 
    geneSymbol = markerGenes, 
    upstream = 50000,
    downstream = 50000,
    loops = getPeak2GeneLinks(proj)
)

ArchR logging to : ArchRLogs/ArchR-plotBrowserTrack-35044e7ab72f4f-Date-2025-09-20_Time-16-40-00.319916.log
If there is an issue, please report to github with logFile!

2025-09-20 16:40:00.420833 : Validating Region, 0.002 mins elapsed.



GRanges object with 6 ranges and 2 metadata columns:
      seqnames              ranges strand |     gene_id      symbol
         <Rle>           <IRanges>  <Rle> | <character> <character>
  [1]     chr6   43770184-43786487      + |        7422       VEGFA
  [2]     chr2 205681990-205798133      + |        8828        NRP2
  [3]     chr5 140332843-140346631      - |        1839       HBEGF
  [4]     chr7   55019021-55256620      + |        1956        EGFR
  [5]     chr3 172505508-172523507      - |        8743     TNFSF10
  [6]     chr8   23020133-23069179      - |        8795   TNFRSF10B
  -------
  seqinfo: 24 sequences from hg38 genome


2025-09-20 16:40:00.502579 : Adding Bulk Tracks (1 of 6), 0.003 mins elapsed.

2025-09-20 16:40:03.006994 : Adding Feature Tracks (1 of 6), 0.045 mins elapsed.

2025-09-20 16:40:03.09641 : Adding Loop Tracks (1 of 6), 0.046 mins elapsed.

2025-09-20 16:40:03.310204 : Adding Gene Tracks (1 of 6), 0.05 mins elapsed.

2025-09-20 16:40:03.58377 : Plotting, 0.054 mins elapsed.

2025-09-20 16:40:04.183793 : Adding Bulk Tracks (2 of 6), 0.064 mins elapsed.

2025-09-20 16:40:06.968425 : Adding Feature Tracks (2 of 6), 0.111 mins elapsed.

2025-09-20 16:40:07.067956 : Adding Loop Tracks (2 of 6), 0.112 mins elapsed.

2025-09-20 16:40:07.623325 : Adding Gene Tracks (2 of 6), 0.122 mins elapsed.

2025-09-20 16:40:07.891841 : Plotting, 0.126 mins elapsed.

2025-09-20 16:40:08.352001 : Adding Bulk Tracks (3 of 6), 0.134 mins elapsed.

2025-09-20 16:40:10.676902 : Adding Feature Tracks (3 of 6), 0.173 mins elapsed.

2025-09-20 16:40:10.747387 : Adding Loop Tracks (3 of 6), 0.174 mins elapsed.

2025-

In [26]:
plotPDF(plotList = p, 
    name = "Plot-Tracks-Marker-Genes-with-Peak2GeneLinks.pdf", 
    ArchRProj = proj, 
    addDOC = FALSE, width = 5, height = 5)

Plotting Gtable!



NULL


Plotting Gtable!



NULL


Plotting Gtable!



...

NULL


Plotting Gtable!



NULL


Plotting Gtable!



NULL
